Auther: Beihui Hu

In [ ]:
import rasterio  
import rasterio.mask
import geopandas as gps
import os
import time
import shutil
from shapely.geometry import box

In [ ]:
auxiliary_data_dir=r''
land_polygon_dir=r''
base_dir=r''
output_prefix='pre_'
raw_prediction_dir=os.path.join(base_dir,'tif')
after_land_mask_dir=os.path.join(base_dir,'after_land_mask')
after_river_mask_dir=os.path.join(base_dir,'after_river_mask')

## 1. land mask

In [ ]:
start_id=1
end_id=start_id+50
start=time.time()
for index in range(start_id,end_id):  
    raw_pre_dir=os.path.join(raw_prediction_dir,f'{output_prefix}{index}.tif')
    out_pre_dir=os.path.join(after_land_mask_dir,f'{output_prefix}{index}_alm.tif')
    raw_prediction_image=rasterio.open(raw_pre_dir)
    bbox = box(*raw_prediction_image.bounds)
    meta=raw_prediction_image.meta
    meta.update({"compress": 'lzw'})
    land_mask = gps.read_file(land_polygon_dir, bbox=bbox) 
    mask_patch_geojson = land_mask.geometry.values 
 # perform land mask
    if len(mask_patch_geojson) > 0:
        modelpredtictions_masked, out_meta = rasterio.mask.mask(raw_prediction_image, mask_patch_geojson, all_touched=True, invert=False)
        print(f'Mask finished: {index} / {end_id-1}')
        with rasterio.open(out_pre_dir, "w", **meta) as prediction_masked:
            prediction_masked.write(modelpredtictions_masked)
    
end=time.time()
print(f'time:{end-start}')

## 2. river mask

In [ ]:
start_id=1
end_id=start_id+50
start=time.time()
for index in range(start_id,end_id):  
    raw_pre_dir=os.path.join(after_land_mask_dir,f'{output_prefix}_{index}_alm.tif')
    out_pre_dir=os.path.join(after_river_mask_dir,f'{output_prefix}_{index}_arm.tif')
    if os.path.exists(raw_pre_dir):
        raw_prediction_image=rasterio.open(raw_pre_dir)
        bbox = box(*raw_prediction_image.bounds)
        meta=raw_prediction_image.meta
        meta.update({"compress": 'lzw'})
        grwl_patch = gps.read_file(auxiliary_data_dir,layer='GRWL_DNRiver',bbox=bbox) 
        osm_patch = gps.read_file(auxiliary_data_dir,layer='osm_natural_water_river',bbox=bbox) 
        river_mask=grwl_patch._append(osm_patch)
        mask_patch_geojson = river_mask.geometry.values 
     # perform land mask
        if len(mask_patch_geojson) > 0:
            modelpredtictions_masked, out_meta = rasterio.mask.mask(raw_prediction_image, mask_patch_geojson, all_touched=True, invert=True)
            print(f'Mask finished: {index} / {end_id-1}')
            with rasterio.open(out_pre_dir, "w", **meta) as prediction_masked:
                prediction_masked.write(modelpredtictions_masked)
        else:
            shutil.copy(raw_pre_dir,out_pre_dir)

end=time.time()
print(f'time:{end-start}')